In [10]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run[0:-2]

dirs = [
    '/home/chesetti/Repos/learned_merge_cleanup/sponge/join_all',
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.dataset',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.key_type,spec.load_sstable_in_mem,spec.name,spec.num_threads,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,spec.index.dataset,spec.index.leaf_size_in_pages
0,sponge/join_all/build/benchmark_runner sponge/...,fb,58845E6C5F877953D258D1EF888CF8AB,983619960,0.983620,386222,8192,3163930608,1990116,7774,...,uint64,False,lsj_sampledflatpgm256100run_2,1,sponge/join_all/fb_1/inputs/input100,sponge/join_all/fb_1/outputs/lsj_sampledflatpg...,8,True,NaN,NaN
1,sponge/join_all/build/benchmark_runner sponge/...,fb,1E3143ADAA2C76210987830B1A42E0D7,3750760614,3.750761,781250,4096,3199999984,19032102,74345,...,uint64,False,sort_join10run_0,1,sponge/join_all/fb_1/inputs/input10,sponge/join_all/fb_1/outputs/sort_join_run_0_r...,8,True,NaN,NaN
2,sponge/join_all/build/benchmark_runner sponge/...,fb,197D7E88032185A864E14627A72FA7A5,26582024874,26.582025,195313,16384,3199999984,199999999,781250,...,uint64,False,lsj_radixspline10241run_0,1,sponge/join_all/fb_1/inputs/input1,sponge/join_all/fb_1/outputs/lsj_radixspline10...,8,True,NaN,NaN
3,sponge/join_all/build/benchmark_runner sponge/...,fb,2644F6D546B276DDC9835D6A19DC3570,4307375729,4.307376,780779,4096,3198070784,19033337,74349,...,uint64,False,lsj_radixspline25610run_2,1,sponge/join_all/fb_1/inputs/input10,sponge/join_all/fb_1/outputs/lsj_radixspline25...,8,True,NaN,NaN
4,sponge/join_all/build/benchmark_runner sponge/...,fb,F247BBF5C744D5062BA4E9E6F328A430,933100377,0.933100,194575,16384,3187912064,1989995,7774,...,uint64,False,lsj_radixspline1024100run_1,1,sponge/join_all/fb_1/inputs/input100,sponge/join_all/fb_1/outputs/lsj_radixspline10...,8,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,sponge/join_all/build/benchmark_runner sponge/...,wiki,5595BF463A4B64A2309EB204E2213780,478566258,0.478566,299728,4096,1227685296,1968806,7691,...,uint64,False,lsj_rmi100run_0,1,sponge/join_all/wiki_1/inputs/input100,sponge/join_all/wiki_1/outputs/lsj_rmi_run_0_r...,8,True,wiki_ts_200M_uint64,NaN
428,sponge/join_all/build/benchmark_runner sponge/...,wiki,2C02CEC7EFAFD3DD40BC876109604EF7,3461193710,3.461194,88316,16384,1446965968,17265174,67443,...,uint64,False,lsj_radixspline102410run_1,1,sponge/join_all/wiki_1/inputs/input10,sponge/join_all/wiki_1/outputs/lsj_radixspline...,8,True,NaN,NaN
429,sponge/join_all/build/benchmark_runner sponge/...,wiki,76667E637BD8D73A3D271483F8B1D5BC,772735806,0.772736,176405,8192,1445105968,1969110,7692,...,uint64,False,lsj_sampledflatpgm256100run_0,1,sponge/join_all/wiki_1/inputs/input100,sponge/join_all/wiki_1/outputs/lsj_sampledflat...,8,True,NaN,NaN
430,sponge/join_all/build/benchmark_runner sponge/...,wiki,4855EC5C974FBDCD0911D950AB48587F,944145288,0.944145,88099,16384,1443399408,1969107,7692,...,uint64,False,lsj_radixspline1024100run_2,1,sponge/join_all/wiki_1/inputs/input100,sponge/join_all/wiki_1/outputs/lsj_radixspline...,8,True,NaN,NaN


In [11]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")


┌──────────────────────┬─────────┬──────────────────────┬───┬─────────┬─────────────────────┬────────────┐
│       command        │ dataset │   result.checksum    │ … │ epsilon │ inner_bytes_fetched │ index_type │
│       varchar        │ varchar │       varchar        │   │ double  │        int64        │  varchar   │
├──────────────────────┼─────────┼──────────────────────┼───┼─────────┼─────────────────────┼────────────┤
│ sponge/join_all/bu…  │ fb      │ 58845E6C5F877953D2…  │ … │   256.0 │          3163930608 │ PGM        │
│ sponge/join_all/bu…  │ fb      │ 1E3143ADAA2C762109…  │ … │    NULL │          3199999984 │ NA         │
│ sponge/join_all/bu…  │ fb      │ 197D7E88032185A864…  │ … │  1024.0 │          3199999984 │ NA         │
│ sponge/join_all/bu…  │ fb      │ 2644F6D546B276DDC9…  │ … │   256.0 │          3198070784 │ NA         │
│ sponge/join_all/bu…  │ fb      │ F247BBF5C744D5062B…  │ … │  1024.0 │          3187912064 │ NA         │
│ sponge/join_all/bu…  │ fb      │ AF

In [12]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='fb'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='books'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='osm'"
    " AND (algo='lsj_pgm256' OR algo='lsj_flatpgm256' OR algo='lsj_sampledpgm256' OR algo='lsj_sampledflatpgm256')"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))



algo,lsj_btree256,lsj_flatpgm256,lsj_pgm256,lsj_radixspline256,lsj_sampledflatpgm256,sort_join
ratio,,,,,,
1,42.300910,22.431856,27.055335,25.670175,23.229437,11.704229
10,5.834207,3.450734,4.009025,4.307376,3.152880,3.904735
100,1.571960,0.940489,1.313947,1.462793,0.983620,2.998804
1000,0.319080,0.243864,0.343778,0.374326,0.385151,2.560174


algo,lsj_flatpgm256,lsj_pgm256,lsj_sampledflatpgm256
ratio,,,
1,22.431856,27.055335,23.229437
10,3.450734,4.009025,3.152880
100,0.940489,1.313947,0.983620
1000,0.243864,0.343778,0.385151


algo,lsj_radixspline1024,sort_join
ratio,,
1,10.778464,5.129859
10,3.267205,2.262956
100,0.926267,1.313295
1000,0.414278,1.190710


algo,lsj_flatpgm256,lsj_pgm256,lsj_sampledflatpgm256
ratio,,,
1,9.457123,11.353547,10.117874
10,2.501715,3.032850,3.073691
100,0.561450,0.750796,0.698167
1000,0.208535,0.317465,0.213931


algo,lsj_radixspline4096,sort_join
ratio,,
1,103.147400,47.431663
10,15.031332,15.704701
100,3.462918,11.254792
1000,2.263213,11.304608


algo,lsj_flatpgm256,lsj_pgm256,lsj_sampledflatpgm256
ratio,,,
1,82.983117,117.044278,88.978646
10,12.694236,16.350766,13.171777
100,3.748309,3.820507,3.968178
1000,1.110650,1.380077,1.210211


In [13]:
rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='256') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='1024') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))

rows = duckdb.sql(
    " SELECT ratio, algo, MEDIAN(inner_disk_fetch) as v FROM df"
    f" WHERE threads=1 AND (algo='hash_join' OR algo='sort_join' or epsilon='4096') AND dataset='wiki'"
    " GROUP BY dataset, threads, ratio, algo"
    " ORDER BY ratio, threads"
).df()
display(rows.pivot(index='ratio', values='v', columns='algo'))


algo,lsj_btree256,lsj_flatpgm256,lsj_pgm256,lsj_radixspline256,lsj_sampledflatpgm256,sort_join
ratio,,,,,,
1,353270.0,176635.0,176635.0,353270.0,176635.0,353270.0
10,353270.0,176611.0,176610.0,353209.0,176635.0,353270.0
100,348811.0,172933.0,172958.0,345715.0,176405.0,353270.0
1000,149418.0,102884.0,102878.0,194161.0,118522.0,353270.0


algo,lsj_radixspline1024,sort_join
ratio,,
1,88318.0,353270.0
10,88316.0,353270.0
100,88099.0,353270.0
1000,79616.0,353270.0


algo,lsj_radixspline4096,sort_join
ratio,,
1,22080.0,353270.0
10,22080.0,353270.0
100,22079.0,353270.0
1000,21960.0,353270.0


In [14]:
def print_for_dataset(dataset, algos):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    print(rows[algos].to_markdown())
    rows[algos].to_csv('all_indexes/'+dataset+'.csv')



algos = ['lsj_sampledflatpgm256', 'lsj_btree256',  'sort_join']
#print_for_dataset("fb", algos)
#print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)



## osm
|   ratio |   lsj_sampledflatpgm256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|---------------:|------------:|
|       1 |                88.9786  |      175.609   |     47.7724 |
|      10 |                13.1718  |       22.2175  |     15.3804 |
|     100 |                 3.96818 |        4.68381 |     11.4066 |
|    1000 |                 1.21021 |        1.11332 |     10.7698 |
## books
|   ratio |   lsj_sampledflatpgm256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|---------------:|------------:|
|       1 |                87.8691  |      174.423   |     47.4317 |
|      10 |                13.3788  |       22.2364  |     15.7047 |
|     100 |                 3.54946 |        4.84053 |     11.2548 |
|    1000 |                 1.20184 |        1.27291 |     11.3046 |


In [34]:
algos = ['lsj_sampledflatpgm256', 'lsj_flatpgm256', 'lsj_radixspline256', 'lsj_rmi' , 'lsj_btree256', 'sort_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)


## fb
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_rmi |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|----------:|---------------:|------------:|
|       1 |               23.2294   |        22.4319   |            25.6702   | 20.3287   |       42.3009  |    11.7042  |
|      10 |                3.68178  |         3.45073  |             4.30738  |  3.76312  |        5.83421 |     4.45029 |
|     100 |                0.98362  |         1.01192  |             1.46279  |  1.25237  |        1.57196 |     3.12833 |
|    1000 |                0.385151 |         0.353513 |             0.374326 |  0.275802 |        0.31908 |     2.86744 |
## wiki
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|---------------:|------------:|
|       1 |               

In [29]:
algos = ['lsj_radixspline256', 'lsj_radixspline1024', 'lsj_radixspline4096']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)

## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"